In [37]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import Quine_McCluskey_Functions as qm

In [38]:
df = pd.read_csv("DFF_result.csv")

# Clean data.

# Pulse width values for Pass cases & Current percentage values for Fail cases. 
df.loc[df['fail'] == 'pass', 'Junction Values'] = df.loc[df['fail'] == 'pass', 'Pulse width info']
df.loc[df['fail'] == 'fail', 'Junction Values'] = df.loc[df['fail'] == 'fail', 'Current percentage']


# One-hot encode pass/fail.
drop_col = ['cus', 'real_flux', 'Faulty Junction', 'Pulse magnitude info', 'Phase state info', 'Pulse width info', 'Current percentage' ]
df = df.drop(drop_col, axis=1)

# df.head()

In [39]:
# Rule 1: Initial Bias is between 60%-80%. 
# Rule 2: Junction 4 bias <55%. 


# Create rule 1, rule 2, pass/fail truth table.
for val in df['Junction Values']:
    val = eval(str(val))
    if 0.6 <= (val['B01RX1']) <= 0.8:
        df['Rule 1'] = 1
    else:
        df['Rule 1'] = 0
    if val['B4'] <0.55:
        df['Rule 2'] = 1
    else:
        df['Rule 2'] = 0

columns = df.columns.tolist()
columns.append(columns.pop(3))
df["fail"] = df["fail"].replace({"pass": 1, "fail": 0})
df = df[columns]

# df.head()

In [40]:
# Create buckets.
condition0 = (df[['Rule 1', 'Rule 2', 'fail']] == [0, 0, 0]).all(axis=1)
condition0_df = df[condition0]
# print(len(condition0_df))

condition1 = (df[['Rule 1', 'Rule 2', 'fail']] == [0, 0, 1]).all(axis=1)
condition1_df = df[condition1]
# print(len(condition1_df))

condition2 = (df[['Rule 1', 'Rule 2', 'fail']] == [0, 1, 0]).all(axis=1)
condition2_df = df[condition2]
# print(len(condition2_df))

condition3 = (df[['Rule 1', 'Rule 2', 'fail']] == [0, 1, 1]).all(axis=1)
condition3_df = df[condition3]
# print(len(condition3_df))

condition4 = (df[['Rule 1', 'Rule 2', 'fail']] == [1, 0, 0]).all(axis=1)
condition4_df = df[condition4]
# print(len(condition4_df))

condition5 = (df[['Rule 1', 'Rule 2', 'fail']] == [1, 0, 1]).all(axis=1)
condition5_df = df[condition5]
# print(len(condition5_df))

condition6 = (df[['Rule 1', 'Rule 2', 'fail']] == [1, 1, 0]).all(axis=1)
condition6_df = df[condition6]
# print(len(condition6_df))

condition7 = (df[['Rule 1', 'Rule 2', 'fail']] == [1, 1, 1]).all(axis=1)
condition7_df = df[condition7]
# print(len(condition7_df))

In [44]:
condition6_df['driver'] = condition6_df['config'].str.split('_').str[0]
condition6_df['load'] = condition6_df['config'].str.split('_').str[6]
condition6_df['splitterNum'] = condition6_df['config'].str.split('_').str[2]
condition6_df = pd.get_dummies(condition6_df, columns = ["driver", "load", "splitterNum"])

truthTable_6 = condition6_df.iloc[:, 9:]

minterms_6 = []
for row in truthTable_6.values:
    binary = "".join(map(str, row))
    minterms_6.append(binary)

minterms_6 = list(set(minterms_6))
minterms_6.sort()
print (len(minterms_6))
no_of_variable = 14
prime_implicants = qm.check(minterms_6)
print("Prime Implicants are:")
print(prime_implicants)
chart = qm.prime_implicant_chart(prime_implicants, minterms_6)

essential_prime_implicants = qm.selection(chart, prime_implicants)
print("Essential Prime Implicants are:")
print(essential_prime_implicants)
print(len(essential_prime_implicants))

100
Prime Implicants are:
['0000100001000100001000010001', '0000100001001000001000010010', '0000100001010000001000010100', '0000100001100000001000011000', '0000100010000100001000100001', '0000100010001000001000100010', '0000100010010000001000100100', '0000100010100000001000101000', '0000100100000100001001000001', '0000100100001000001001000010', '0000100100010000001001000100', '0000100100100000001001001000', '0000101000000100001010000001', '0000101000001000001010000010', '0000101000010000001010000100', '0000101000100000001010001000', '0000110000000100001100000001', '0000110000001000001100000010', '0000110000010000001100000100', '0000110000100000001100001000', '0001000001000100010000010001', '0001000001001000010000010010', '0001000001010000010000010100', '0001000001100000010000011000', '0001000010000100010000100001', '0001000010001000010000100010', '0001000010010000010000100100', '0001000010100000010000101000', '0001000100000100010001000001', '0001000100001000010001000010', '000100010001

In [43]:
condition7_df['driver'] = condition7_df['config'].str.split('_').str[0]
condition7_df['load'] = condition7_df['config'].str.split('_').str[6]
condition7_df['splitterNum'] = condition7_df['config'].str.split('_').str[2]
condition7_df = pd.get_dummies(condition7_df, columns = ["driver", "load", "splitterNum"])

truthTable_7 = condition7_df.iloc[:, 9:]

minterms_7 = []
for row in truthTable_7.values:
    binary = "".join(map(str, row))
    minterms_7.append(binary)

minterms_7 = list(set(minterms_7))
minterms_7.sort()
print(len(minterms_7))

no_of_variable = 14
prime_implicants = qm.check(minterms_7)
print("Prime Implicants are:")
print(prime_implicants)
chart = qm.prime_implicant_chart(prime_implicants, minterms_7)

essential_prime_implicants = qm.selection(chart, prime_implicants)
print("Essential Prime Implicants are:")
print(essential_prime_implicants)
print(len(essential_prime_implicants))

75
Prime Implicants are:
['00001000010010000100001001', '00001000010100000100001010', '00001000011000000100001100', '00001000100010000100010001', '00001000100100000100010010', '00001000101000000100010100', '00001001000010000100100001', '00001001000100000100100010', '00001001001000000100100100', '00001010000010000101000001', '00001010000100000101000010', '00001010001000000101000100', '00001100000010000110000001', '00001100000100000110000010', '00001100001000000110000100', '00010000010010001000001001', '00010000010100001000001010', '00010000011000001000001100', '00010000100010001000010001', '00010000100100001000010010', '00010000101000001000010100', '00010001000010001000100001', '00010001000100001000100010', '00010001001000001000100100', '00010010000010001001000001', '00010010000100001001000010', '00010010001000001001000100', '00010100000010001010000001', '00010100000100001010000010', '00010100001000001010000100', '00100000010010010000001001', '00100000010100010000001010', '0010000001100